In [2]:
from sklearn.datasets import make_classification
import torch

In [ ]:
# step1 : create a synthetic classification dataset using sklearn
X,y = make_classification(
    n_samples = 10, # no. of samples
    n_features= 2, # nu. of features
    n_informative= 2, # number of informative features
    n_redundant= 0, # no. of redundant features
    n_classes = 2, # no. of classes
    random_state= 42 # for reproducibility
)

In [4]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [9]:
X = torch.tensor(X, dtype = torch.float32)
y = torch.tensor(y, dtype = torch.long)

In [10]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [11]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [12]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset): # inheriting Dataset class
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
        # data transformation is done in __getitem__
        

In [14]:
dataset = CustomDataset(X,y)

In [15]:
len(dataset)

10

In [21]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [22]:
dataloader = DataLoader(dataset, batch_size= 2, shuffle= True)

In [23]:
for batch_features , batch_lables in dataloader:
    print(batch_features)
    print(batch_lables)
    print("-"*50)

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388]])
tensor([1, 0])
--------------------------------------------------
tensor([[ 1.7774,  1.5116],
        [-0.9382, -0.5430]])
tensor([1, 1])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-0.5872, -1.9717]])
tensor([0, 0])
--------------------------------------------------
tensor([[-0.7206, -0.9606],
        [ 1.7273, -1.1858]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.8997,  0.8344],
        [-2.8954,  1.9769]])
tensor([1, 0])
--------------------------------------------------


*READ PARALELLIZATION*

### Now, Improving our Breast Cancer Detection model


In [24]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [26]:
df.shape

(569, 33)

In [27]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [28]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## train_test_split

In [29]:
X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size = 0.2)

### scaling

In [30]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [31]:
X_train

array([[-1.06055119e+00, -9.86787759e-01, -1.03178034e+00, ...,
        -3.38195402e-01,  2.25932140e-01,  7.26535056e-01],
       [ 3.75588263e-02, -1.33467650e+00,  1.41036916e-03, ...,
        -9.71538418e-02,  2.97500287e-01, -1.26812912e+00],
       [ 1.22904164e-01, -5.55958660e-01,  7.16111765e-02, ...,
        -5.67184884e-01, -7.36829843e-01, -9.75682554e-01],
       ...,
       [ 3.56181421e-01,  4.99667574e-02,  3.80907675e-01, ...,
         5.62620172e-01, -8.59005022e-02, -1.81500371e-01],
       [ 1.88335590e-01, -5.97428841e-01,  2.54546221e-01, ...,
         1.35527607e+00,  1.18869412e+00,  1.28687161e+00],
       [ 3.76095333e-01,  3.29385644e+00,  4.63496860e-01, ...,
         1.33518928e+00,  1.06430187e+00,  2.06263635e+00]])

In [32]:
y_train

426    B
225    B
486    B
545    B
260    M
      ..
441    M
86     M
131    M
117    M
259    M
Name: diagnosis, Length: 455, dtype: object

### Label Encoding


In [33]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [34]:
y_train

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,

### Numpy arrays to PyTorch tensors

In [74]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [75]:
X_train_tensor.dtype

torch.float64

In [92]:
X_train_tensor = X_train_tensor.float()
y_train_tensor = y_train_tensor.float()
X_test_tensor = X_test_tensor.float()
y_test_tensor = y_test_tensor.float()

In [80]:
y_train_tensor.dtype

torch.float32

### Creating custom class

In [93]:
class CustomDataset2(Dataset):
    def __init__(self,features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self,idx):
        return self.features[idx] , self.labels[idx]
        

In [94]:
train_dataset = CustomDataset2(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset2(X_test_tensor, y_test_tensor)

In [95]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [96]:
train_dataset[0][1].dtype

torch.float32

### Defining the model

In [97]:
import torch.nn as nn

class MySimpleNN(nn.Module):
    def __init__(self,num_features):
        
        super().__init__()
        
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid() 
        
    def forward(self,features):
        
        out = self.linear(features)
        out = self.sigmoid(out)
        
        return out    

### Important parameters


In [98]:
learning_rate = 0.1
epochs = 25

In [99]:
# define loss function
loss_function = nn.BCELoss()

### Training Pipeline
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)



In [100]:
# define loop
for i in range(epochs):
    
    for batch_features, batch_labels in train_loader:
        
        # forward pass
        y_pred = model(batch_features)
        
        # loss calculate
        loss = loss_function(y_pred, batch_labels.view(-1,1))
        
        # it is recommended to cleaer gradient before backward pass
        optimizer.zero_grad()
        
        # backward pass
        loss.backward()
        
        # parameters update1
        optimizer.step()
        
    # print loss in each epoch
    print(f"Epoch : {i + 1} , Loss : {loss.item()}")
        
        

Epoch : 1 , Loss : 0.1802920252084732
Epoch : 2 , Loss : 0.23156610131263733
Epoch : 3 , Loss : 0.05965812876820564
Epoch : 4 , Loss : 0.10010696947574615
Epoch : 5 , Loss : 0.054986052215099335
Epoch : 6 , Loss : 0.00893841590732336
Epoch : 7 , Loss : 0.6970844864845276
Epoch : 8 , Loss : 0.05404861643910408
Epoch : 9 , Loss : 0.08907948434352875
Epoch : 10 , Loss : 0.10913018137216568
Epoch : 11 , Loss : 0.10523469001054764
Epoch : 12 , Loss : 0.1349572390317917
Epoch : 13 , Loss : 0.11400572955608368
Epoch : 14 , Loss : 0.029087955132126808
Epoch : 15 , Loss : 0.10591982305049896
Epoch : 16 , Loss : 0.04382767900824547
Epoch : 17 , Loss : 0.04085707291960716
Epoch : 18 , Loss : 0.2103722095489502
Epoch : 19 , Loss : 0.15434400737285614
Epoch : 20 , Loss : 0.007719065528362989
Epoch : 21 , Loss : 0.011948302388191223
Epoch : 22 , Loss : 0.09453921765089035
Epoch : 23 , Loss : 0.012404278852045536
Epoch : 24 , Loss : 0.14067348837852478
Epoch : 25 , Loss : 0.19252073764801025


In [101]:
model.linear.weight

Parameter containing:
tensor([[ 0.5935,  0.6639,  0.5122,  0.4446,  0.1296,  0.1084,  0.4245,  0.6591,
          0.1415, -0.1522,  0.6325, -0.0098,  0.5620,  0.3799,  0.0807, -0.3372,
         -0.1581,  0.0807, -0.2236, -0.3449,  0.6336,  0.5744,  0.6031,  0.6943,
          0.4194,  0.3668,  0.4629,  0.6358,  0.5138,  0.0406]],
       requires_grad=True)

In [102]:
model.linear.bias

Parameter containing:
tensor([-0.3429], requires_grad=True)

### Evaluation

In [103]:
# model evaluation
model.eval() # set the model to evaluation mode
accuracy_list = []


with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # forward pass
        
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()
        
        # calculate accuracy for current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)    
        
# overall accuracy
overall_accuracy = sum(accuracy_list)/len(accuracy_list)
print(f"Accuracy : {overall_accuracy:.4f}")    

    

Accuracy : 0.9705
